In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization, Dense, Activation, Dropout, Input, LSTM, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

/share/software/user/open/py-scipy/1.6.3_py39/lib/python3.9/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "
2025-11-30 13:20:10.650809: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-30 13:20:10.847250: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-11-30 13:20:14.402641: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.

## Scaling all sfe and climate data to be between 0 and 1
https://www.geeksforgeeks.org/deep-learning/long-short-term-memory-lstm-rnn-in-tensorflow/
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html

In [2]:
# read in list of arrays for each pixel - these are fully filtered
sfe_pre = np.load("/scratch/users/ashdef/pre_treatment_data/no_nans/sfe_filtered_3mwindow.npz", allow_pickle = True)["sfe_window"] 
climate_pre = np.load("/scratch/users/ashdef/pre_treatment_data/no_nans/climate_filtered_3mwindow.npz", allow_pickle = True)["climate_window"]



In [3]:
print(sfe_pre.shape)
print(climate_pre.shape)

(3836373,)
(3836373, 3, 8)


In [4]:
sfe_pre_rs = sfe_pre.reshape(-1,1) # single feature
sfe_pre_rs.shape


(3836373, 1)

In [5]:
# climate data is 3d, needs to be 2d for minmaxscaler
# so reshape and then turn back to original shape after processing
climate_pre_ogshape = climate_pre.shape
print(climate_pre_ogshape)

climate_pre_rs = climate_pre.reshape(-1, climate_pre_ogshape[2]) # (pixel-month pairs * timesteps, 8)
climate_pre_rs.shape

(3836373, 3, 8)


(11509119, 8)

In [6]:
scaler_sfe = MinMaxScaler(feature_range=(0,1))
scaler_climate = MinMaxScaler(feature_range=(0,1))

sfe_pre_scaled = scaler_sfe.fit_transform(sfe_pre_rs)
climate_pre_scaled = scaler_climate.fit_transform(climate_pre_rs)



In [7]:
print(sfe_pre_scaled.min(), sfe_pre_scaled.max())
print(climate_pre_scaled.min(), climate_pre_scaled.max())


0.0 1.0
0.0 1.0000000000000002


In [8]:
sfe_back = scaler_sfe.inverse_transform(sfe_pre_scaled)

sfe_back.shape
print(sfe_back.min(), sfe_back.max())
print(sfe_pre_rs.min(), sfe_pre_rs.max())


-0.4033722536155057 5.223653647385592
-0.40337225361550577 5.223653647385593


In [9]:
# change climate data back to orig shape
climate_pre_scaled = climate_pre_scaled.reshape(climate_pre_ogshape)

In [10]:
climate_pre_scaled = np.array(climate_pre_scaled)
sfe_pre_scaled = np.array(sfe_pre_scaled)

# wrap the lists as object arrays - allows arrays of different lengths to be saved
sfe_obj = np.array(sfe_pre_scaled, dtype=object)
climate_obj = np.array(climate_pre_scaled, dtype=object)

In [11]:
np.savez_compressed("/scratch/users/ashdef/pre_treatment_data/no_nans/sfe_3mwindow_scaled.npz", sfe_pre = sfe_obj)
np.savez_compressed("/scratch/users/ashdef/pre_treatment_data/no_nans/climate_3mwindow_scaled.npz", climate_pre =climate_obj)

## training the model

https://www.geeksforgeeks.org/deep-learning/long-short-term-memory-lstm-rnn-in-tensorflow/

In [12]:
# reading in sliding window data, was also scaled
sfe_pre = np.load("/scratch/users/ashdef/pre_treatment_data/no_nans/sfe_3mwindow_scaled.npz", allow_pickle = True)["sfe_pre"] 
climate_pre = np.load("/scratch/users/ashdef/pre_treatment_data/no_nans/climate_3mwindow_scaled.npz", allow_pickle = True)["climate_pre"]

In [13]:
# converting from objects
sfe_pre = np.array(sfe_pre, dtype=np.float32)
climate_pre = np.array(climate_pre, dtype=np.float32)


In [14]:
climate_train, climate_test, sfe_train, sfe_test = train_test_split(
    climate_pre, sfe_pre, test_size=0.1, shuffle=False
)

In [15]:
model = Sequential()
model.add(LSTM(units=64, return_sequences=False,
          input_shape=(climate_pre.shape[1], climate_pre.shape[2]))) # return one output per sequence
model.add(Dropout(0.2))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse', metrics = ['mae'])
model.summary()

2025-11-30 13:21:29.066324: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-30 13:21:29.568487: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78910 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:c0:00.0, compute capability: 8.0


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                18688     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 18,753
Trainable params: 18,753
Non-trainable params: 0
_________________________________________________________________


In [16]:
history = model.fit(climate_train, sfe_train, epochs=40, batch_size=64, validation_split=0.1)


Epoch 1/40


2025-11-30 13:21:35.699647: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101
2025-11-30 13:21:38.570640: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


48555/48555 [==============================] - 189s 4ms/step - loss: 0.0032 - mae: 0.0417 - val_loss: 0.0026 - val_mae: 0.0383
Epoch 2/40
48555/48555 [==============================] - 183s 4ms/step - loss: 0.0024 - mae: 0.0364 - val_loss: 0.0024 - val_mae: 0.0355
Epoch 3/40
48555/48555 [==============================] - 183s 4ms/step - loss: 0.0023 - mae: 0.0351 - val_loss: 0.0024 - val_mae: 0.0356
Epoch 4/40
48555/48555 [==============================] - 185s 4ms/step - loss: 0.0022 - mae: 0.0343 - val_loss: 0.0020 - val_mae: 0.0324
Epoch 5/40
48555/48555 [==============================] - 184s 4ms/step - loss: 0.0021 - mae: 0.0337 - val_loss: 0.0021 - val_mae: 0.0332
Epoch 6/40
48555/48555 [==============================] - 185s 4ms/step - loss: 0.0020 - mae: 0.0333 - val_loss: 0.0019 - val_mae: 0.0311
Epoch 7/40
48555/48555 [==============================] - 184s 4ms/step - loss: 0.0020 - mae: 0.0330 - val_loss: 0.0019 - val_mae: 0.0317
Epoch 8/40
48555/48555 [=====================

In [17]:
model.save("/scratch/users/ashdef/model_out/3month_no_batchnorm/baseline.keras")


In [18]:
history.history

{'loss': [0.0031971524003893137,
  0.0024241586215794086,
  0.0022640270181000233,
  0.0021671385038644075,
  0.0020971244666725397,
  0.0020445638801902533,
  0.00199940986931324,
  0.001962085021659732,
  0.001928896876052022,
  0.0019007324008271098,
  0.0018738389480859041,
  0.0018523498438298702,
  0.0018306059064343572,
  0.0018115888815373182,
  0.0017923943232744932,
  0.0017776224995031953,
  0.001761214924044907,
  0.0017461901297792792,
  0.0017335889860987663,
  0.0017208572244271636,
  0.0017094878712669015,
  0.0016995463520288467,
  0.0016900802729651332,
  0.0016796161653473973,
  0.0016698980471119285,
  0.001662969938479364,
  0.0016537929186597466,
  0.0016467756358906627,
  0.0016391514800488949,
  0.0016343413153663278,
  0.0016265331069007516,
  0.001621016301214695,
  0.0016147533897310495,
  0.001610904117114842,
  0.0016033918363973498,
  0.0015989993698894978,
  0.0015930715017020702,
  0.001589593244716525,
  0.0015843064757063985,
  0.0015821533743292093],


In [19]:
np.save("/scratch/users/ashdef/model_out/3month_no_batchnorm/training_history.npy", history.history)

In [20]:
predictions = model.predict(climate_test)
np.save("/scratch/users/ashdef/model_out/3month_no_batchnorm/predictions_scaled.npy", predictions)

predictions_original = scaler_sfe.inverse_transform(predictions)
np.save("/scratch/users/ashdef/model_out/3month_no_batchnorm/predictions_original.npy", predictions_original)

np.save("/scratch/users/ashdef/model_out/3month_no_batchnorm/sfe_test_scaled.npy", sfe_test)
    
sfe_test_original = scaler_sfe.inverse_transform(sfe_test)
np.save("/scratch/users/ashdef/model_out/3month_no_batchnorm/sfe_test_original.npy", sfe_test_original)

11989/11989 [==============================] - 21s 2ms/step


In [21]:
rmse = mean_squared_error(sfe_test_original, predictions_original, squared=False)
mae = mean_absolute_error(sfe_test_original, predictions_original)
r2 = r2_score(sfe_test_original, predictions_original)

print(rmse, mae, r2)

0.21469584 0.15599923 0.9659330225912939
